## Import lab

In [74]:
import cv2 as cv
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout,InputLayer
# from tensorflow.keras.layers import *
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold

# for i in dir(tensorflow.keras.layers):
#     m=i.lower()
#     if m.startswith("i"):
#         print(i)

# constants

In [2]:
g=cv.COLOR_BGR2GRAY
font=cv.FONT_HERSHEY_COMPLEX

## Generate DataSete

In [3]:
def genrateDataset():
    face_classfiler=cv.CascadeClassifier("datasets/haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray=cv.cvtColor(img,g)
        faces=face_classfiler.detectMultiScale(gray,1.3,5)
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face=img[y:y+h,x:x+w]
        return cropped_face
    cap=cv.VideoCapture(0)
    img_id=0
    while True:
        ret,frame=cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face=cv.resize(face_cropped(frame),(200,200))
            face=cv.cvtColor(face,g)
            file_name_path=f"datasets/imges/gamal.{img_id}.jpg"
            cv.imwrite(file_name_path,face)
            cv.putText(face,str(img_id),(50,50),font,1,(0,255,0),2)
            cv.imshow("face",face)
            print(img_id)
            if cv.waitKey(1) & 0xFF ==ord("q") or int(img_id)==20:
                break
    cap.release()
    cv.destroyAllWindows()
    print("End....")
genrateDataset()

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\gamal\AppData\Local\Temp\ipykernel_12596\2600257060.py:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
End....


## Createing Model

In [4]:
def my_label(image_name):
    name=image_name.split('.')[-3]
    if name=="person1":
        return np.array([1,0,0])
    elif name=="person1":
        return np.array([0,1,0])
    elif name=="basma":
        return np.array([0,0,1])

## read images from dir and save in random array

In [5]:
def my_data():
    data=[]
    for img in tqdm(os.listdir("datasets/imges/")):
        path=f"datasets/imges/{img}"
        img_data=cv.imread(path,cv.IMREAD_GRAYSCALE)
        img_data=cv.resize(img_data,(50,50))
        data.append([np.array(img_data),my_label(img)])
    shuffle(data)
    return data


# face=cv.resize(my_data()[0][0],(400,400))
# cv.imshow(str(my_data()[0][1]),face)
# k=cv.waitKey(0)

# if k==27:
#     cv2.destroyAllWindows()

## Create train and test Data

In [13]:
test_size=0.25
data=my_data()
test_dim=int(len(data)* test_size)
traing_dim=(int(len(data)-test_dim))
print(test_dim,traing_dim)

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1364.42it/s]

15 45


In [21]:
traing=data[:traing_dim]
test=data[traing_dim:]
len(traing)+len(test)

60

### create X_train,Y_train ,X_test,Y_test

In [33]:
# print(traing)
X_train=np.array([i[0] for i in traing]).reshape(-1,50,50,1)
Y_train=np.array([i[1] for i in traing])

X_test=np.array([i[0] for i in test]).reshape(-1,50,50,1)
Y_test=np.array([i[1] for i in test])

X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((45, 50, 50, 1), (45, 3), (15, 50, 50, 1), (15, 3))

## start model

In [ ]:
tf.reset_default_graph()
convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet, 32, 5, activation='relu')
# 32 filters and stride=5 so that the filter will move 5 pixel or unit at a time
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossentropy')
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(X_train, y_train, n_epoch=12, validation_set=(X_test, y_test), show_metric = True, run_id="FRS" )

In [ ]:
def data_for_visualization():
    Vdata = []
    for img in tqdm(os.listdir("Images for visualization")):
        path = os.path.join("Images for visualization", img)
        img_num = img.split('.')[0] 
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [ ]:
Vdata = data_for_visualization()


In [ ]:
import matplotlib.pyplot as plt   # pip install matplotlib

fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num+1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 0:
        my_label = 'Ishwar'
    elif np.argmax(model_out) == 1:
        my_label = 'Manish'
    else:
        my_label = 'Bijay'
        
    y.imshow(image, cmap='gray')
    plt.title(my_label)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()